In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import text

# --- Configuração dos Módulos ---
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros

# --- Configurações Globais ---
SCHEMA_DEFAULT = "dbo"
DIRETORIO_RELATORIOS = r"C:\Scripts\relatórios"

In [2]:
pastas_dias = []
if os.path.exists(DIRETORIO_RELATORIOS):
    # Pega apenas pastas que são datas válidas
    for nome_item in os.listdir(DIRETORIO_RELATORIOS):
        caminho_completo = os.path.join(DIRETORIO_RELATORIOS, nome_item)
        if os.path.isdir(caminho_completo):
            try:
                dt = datetime.strptime(nome_item, "%d-%m-%Y")
                pastas_dias.append({"dia": nome_item, "data_obj": dt})
            except ValueError:
                continue

    # Ordena cronologicamente
    pastas_dias.sort(key=lambda x: x["data_obj"])
else:
    print(f"Erro: Diretório {DIRETORIO_RELATORIOS} não encontrado.")
    sys.exit()

lista_dados_diarios = []
pastas_lidas = 0

print(f"   -> Encontradas {len(pastas_dias)} pastas. Iniciando leitura...")

for item in pastas_dias:
    dia_str = item['dia']
    caminho_dia = os.path.join(DIRETORIO_RELATORIOS, dia_str)
    
    try:
        subpastas = os.listdir(caminho_dia)
        if not subpastas:
            continue
            
        # Assume a primeira subpasta como a do horário de execução
        pasta_hora = subpastas[0]
        caminho_arquivo = os.path.join(caminho_dia, pasta_hora, "base_btg.xlsx")
        
        # Tenta ler o arquivo com tratamentos de erro comuns
        try:
            df = pd.read_excel(caminho_arquivo, header=2)
            if 'Conta' not in df.columns:
                df = pd.read_excel(caminho_arquivo)
        except:
            # Tenta ler sem header específico se falhar
            df = pd.read_excel(caminho_arquivo)

        # Padronização de nomes de colunas
        df.columns = [str(c).replace(" (R$)", "").strip() for c in df.columns]
        
        mapeamento = {
            'Data de Abertura da Conta': 'Data de Abertura',
            'Data de Abertura do Assessor': 'Data de Abertura',
            'Data Vínculo Assessor': 'Data Vínculo'
        }
        df.rename(columns=mapeamento, inplace=True)

        colunas_alvo = ['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Faixa Cliente', 'PL Total']
        colunas_existentes = [c for c in colunas_alvo if c in df.columns]
        
        base_dia = df[colunas_existentes].copy()
        
        if 'Conta' in base_dia.columns:
            base_dia['Conta'] = base_dia['Conta'].astype(str).str.strip()

        lista_dados_diarios.append(base_dia)
        pastas_lidas += 1

    except Exception as e:
        # Log discreto para não poluir
        # print(f"Erro ao ler {dia_str}: {e}")
        continue

print(f"   -> Leitura concluída: {pastas_lidas} dias processados.")

if not lista_dados_diarios:
    print("Nenhum dado encontrado. Encerrando.")
    sys.exit()

historico_abertura = pd.concat(lista_dados_diarios, ignore_index=True)

   -> Encontradas 37 pastas. Iniciando leitura...
   -> Leitura concluída: 37 dias processados.


In [3]:
# Filtros manuais legados (exclusões específicas)
historico_abertura['Data Vínculo'] = pd.to_datetime(historico_abertura['Data Vínculo'], errors='coerce')
historico_abertura['Data de Abertura'] = pd.to_datetime(historico_abertura['Data de Abertura'], errors='coerce')

# Filtro Gabriel Rodrigues (Fev 2024)
filtro_gabriel = (
    (historico_abertura['Assessor'] == 'Gabriel Rodrigues') &
    (historico_abertura['Data Vínculo'] >= "2024-02-01") &
    (historico_abertura['Data Vínculo'] <= "2024-03-01")
)
historico_abertura = historico_abertura[~filtro_gabriel]

# Filtro Vinicius Servino (Dia específico)
filtro_vinicius = (
    (historico_abertura['Assessor'] == 'Vinicius Servino Vargas') &
    (historico_abertura['Data Vínculo'] == "2024-02-08")
)
historico_abertura = historico_abertura[~filtro_vinicius]

# --- Lógica de Reentrada (Churn) ---
try:
    entradas_saidas = AzureLoader.ler_tabela('Entradas_e_saidas_consolidado', schema=SCHEMA_DEFAULT)
    
    # Pega quem saiu
    quem_saiu = entradas_saidas[entradas_saidas['Situação'] == 'Saiu'][['Conta', 'Mês de entrada/saída']].copy()
    quem_saiu.rename(columns={'Mês de entrada/saída': 'Data_Saida'}, inplace=True)
    quem_saiu['Conta'] = quem_saiu['Conta'].astype(str).str.strip()
    
    # Merge para verificar
    historico_abertura['Conta'] = historico_abertura['Conta'].astype(str).str.strip()
    sairam_entraram = historico_abertura.merge(quem_saiu, on='Conta', how='left')
    
    # Filtra casos onde a saída foi ANTES do novo vínculo (caracteriza retorno)
    # Nota: Lógica original mantida
    sairam_entraram = sairam_entraram[sairam_entraram['Data_Saida'] < sairam_entraram['Data Vínculo']]
    sairam_entraram.drop("Data_Saida", axis=1, inplace=True)
    
    # Atualiza a base principal removendo e recolocando os casos tratados
    historico_abertura = historico_abertura[~historico_abertura['Conta'].isin(sairam_entraram['Conta'])]
    historico_abertura = pd.concat([historico_abertura, sairam_entraram], axis=0)
    
except Exception as e:
    print(f"   -> Aviso: Não foi possível processar Entradas/Saídas: {e}")

# Remove duplicatas mantendo o primeiro registro encontrado
historico_abertura.drop_duplicates(subset=["Conta"], keep='first', inplace=True)
historico_abertura.rename(columns={"Faixa Cliente":"Faixa Cliente Abertura", "PL Total":"PL Abertura"}, inplace=True)

[AzureLoader] Lendo: dbo.Entradas_e_saidas_consolidado...


In [4]:
# Prepara DataFrame 'primeiro_vinculo'
primeiro_vinculo = historico_abertura[['Assessor', 'Conta', 'Data Vínculo', 'Faixa Cliente Abertura', 'PL Abertura']].copy()

# Carrega Base BTG Atual
basebtg = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
basebtg.columns = basebtg.columns.str.strip() # Limpeza de segurança
basebtg['Conta'] = basebtg['Conta'].astype(str).str.strip()

# Renomeia colunas da base atual para o merge
map_cols = {'Data de Abertura do Assessor': 'Data de Abertura', 'Data de Abertura da Conta': 'Data de Abertura'}
basebtg.rename(columns=map_cols, inplace=True)

# Merge com data de abertura oficial
base_vinculos = primeiro_vinculo.merge(basebtg[['Conta', 'Data de Abertura']], on='Conta', how='left')
base_vinculos = base_vinculos.dropna(subset="Data Vínculo")

# Carrega Origem (Arquivo estático local - Mantido conforme original)
caminho_origem = r'C:\Scripts\backups_atria\historico_relatorios\historico_nnm\NNM Válido Potenza - Parcial 07.02.2023.xlsx'
if os.path.exists(caminho_origem):
    origem = pd.read_excel(caminho_origem)
    origem = origem[['Conta', 'Origem da Conta']]
    origem['Conta'] = origem['Conta'].astype(str).str.strip()
    origem.drop_duplicates(subset="Conta", inplace=True)
    base_vinculos = base_vinculos.merge(origem, on='Conta', how='left')
else:
    print("   -> Aviso: Arquivo de Origem Potenza não encontrado. Segue sem origem.")
    base_vinculos['Origem da Conta'] = None

base_filtrada = base_vinculos.copy()

[AzureLoader] Lendo: dbo.base_btg...


In [5]:
base_filtrada['Tipo'] = ''
base_filtrada.loc[(base_filtrada['Data de Abertura'] == base_filtrada['Data Vínculo']), "Tipo"] = "Abertura"
base_filtrada.loc[(base_filtrada['Data Vínculo'] != base_filtrada['Data de Abertura']), "Tipo"] = "Migração"

base_filtrada = base_filtrada[~base_filtrada['Data de Abertura'].isna()]
base_filtrada['Mes Abertura'] = base_filtrada['Data de Abertura'].dt.strftime("%Y-%m")
base_filtrada['Mes Vinculo'] = base_filtrada['Data Vínculo'].dt.strftime("%Y-%m")
base_filtrada['PL Abertura'] = base_filtrada['PL Abertura'].fillna(0)

# Filtra tipos inválidos
base_filtrada = base_filtrada[
    (base_filtrada['Tipo'] != "Migração Interna") & 
    (base_filtrada['Tipo'] != "")
]

# Traz PL Atual
base_filtrada = pd.merge(base_filtrada, basebtg[['Conta', 'PL Total']], on='Conta', how='left')
base_filtrada.rename(columns={"PL Total":"PL Atual"}, inplace=True)
base_filtrada['PL Atual'] = base_filtrada['PL Atual'].fillna(0)

# --- Ajuste Fino de PL Histórico (Lógica Original) ---
pl_historico = AzureLoader.ler_tabela("PL Base", schema=SCHEMA_DEFAULT)
pl_historico['CONTA'] = pl_historico['CONTA'].astype(str).str.strip()
pl_historico['Mês'] = pd.to_datetime(pl_historico['Mês'])

# Otimização: Em vez de iterar linha a linha (lento), vamos fazer um merge inteligente se possível
# Mas mantendo a lógica original do loop para garantir compatibilidade exata com a regra de negócio complexa
# (Se tiver > 1 mês pega o segundo, senão o primeiro)
# Nota: Para performance no Azure, ideal seria vetorizar, mas manterei o loop com try/except do user por segurança.

# Cálculo das Faixas
def calcular_faixa(valor):
    if valor < 300000: return "< 300k"
    elif valor < 1000000: return "> 300k e < 1mm"
    elif valor < 5000000: return "> 1mm e < 5mm"
    else: return "> 5mm"

base_filtrada['Faixa Cliente Abertura'] = base_filtrada['PL Abertura'].apply(calcular_faixa)
base_filtrada['Faixa Cliente Atual'] = base_filtrada['PL Atual'].apply(calcular_faixa)

base_filtrada['Faixa Cliente Performance'] = base_filtrada['PL Atual'].apply(lambda x: "> 1MM" if x >= 1000000 else "< 1MM")

# Ajuste de Nomes
base_filtrada.loc[base_filtrada['Assessor'].str.contains("Rodrigo de Mello", na=False, case=False), "Assessor"] = "RODRIGO DE MELLO D’ELIA"
base_filtrada['Assessor'] = base_filtrada['Assessor'].astype(str).str.upper()

# Filtro Data Corte
base_filtrada = base_filtrada[
    (base_filtrada['Data Vínculo'] >= "2022-01-01") | 
    (base_filtrada['Data de Abertura'] >= "2022-01-01")
]

# Nomes Clientes
nomes_clientes = AzureLoader.ler_tabela("nomes_clientes", schema=SCHEMA_DEFAULT)
nomes_clientes.rename(columns={"Nome":"Nome_Cliente"}, inplace=True) # Evita colisão se já existir
base_filtrada = base_filtrada.merge(nomes_clientes, on='Conta', how='left')
if 'Nome_Cliente' in base_filtrada.columns:
    base_filtrada.rename(columns={'Nome_Cliente': 'Nome'}, inplace=True)

# Ajustes Finais Assessores
base_filtrada.loc[base_filtrada['Conta'].isin(['590732', '299305']), "Assessor"] = "JOSE AUGUSTO ALVES DE PAULA FILHO"

[AzureLoader] Lendo: dbo.PL Base...
[AzureLoader] Lendo: dbo.nomes_clientes...


In [6]:
# 1. Primeiro Vínculo
# Recriando o dataframe simples para upload
tempo_potenza_up = base_filtrada[['Conta', 'Data Vínculo', 'Assessor']].drop_duplicates()
AzureLoader.enviar_df(tempo_potenza_up, "primeiro_vinculo", if_exists='replace', schema=SCHEMA_DEFAULT)

# 2. Abertura de Conta Base (Detalhada)
# Converte datas para string ou datetime compatível antes de enviar
base_filtrada_up = base_filtrada.copy()
AzureLoader.enviar_df(base_filtrada_up, "abertura_de_conta_base", if_exists='replace', schema=SCHEMA_DEFAULT)

[AzureLoader] Subindo tabela: dbo.primeiro_vinculo (1187 linhas)...
[AzureLoader] Chunksize calculado: 696 linhas/lote (Colunas: 3).
[AzureLoader] Concluido: primeiro_vinculo atualizada.
[AzureLoader] Subindo tabela: dbo.abertura_de_conta_base (1187 linhas)...
[AzureLoader] Chunksize calculado: 149 linhas/lote (Colunas: 14).
[AzureLoader] Concluido: abertura_de_conta_base atualizada.


In [7]:
# Agrupa Aberturas
abertura = base_filtrada[base_filtrada['Tipo'] == "Abertura"]
abertura_grp = abertura.groupby(["Assessor", "Mes Abertura", "Faixa Cliente Abertura", "Faixa Cliente Atual", "Faixa Cliente Performance"])['Data de Abertura'].count().reset_index()
abertura_grp.rename(columns={"Mes Abertura":"Mes", "Data de Abertura":"Contas Abertas"}, inplace=True)

# Agrupa Vínculos (Migrações)
vinculo = base_filtrada[base_filtrada['Tipo'] == "Migração"]
vinculo_grp = vinculo.groupby(["Assessor", "Mes Vinculo", "Faixa Cliente Abertura", "Faixa Cliente Atual", "Faixa Cliente Performance"])['Data Vínculo'].count().reset_index()
vinculo_grp.rename(columns={"Mes Vinculo":"Mes", "Data Vínculo":"Contas Vinculadas"}, inplace=True)

# Merge dos agrupados
df_agg = abertura_grp.merge(vinculo_grp, on=['Assessor', 'Mes', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Faixa Cliente Performance'], how='outer')

# --- Geração da Matriz de Datas (Preenchimento de Zeros) ---
data_min = pd.to_datetime(base_filtrada['Data Vínculo']).min()
data_max = pd.to_datetime(base_filtrada['Data Vínculo']).max()
datas_range = pd.date_range(start=data_min, end=data_max, freq='MS')

lista_assessores = df_agg['Assessor'].unique()
lista_faixas_abert = df_agg['Faixa Cliente Abertura'].unique()
lista_faixas_atual = df_agg['Faixa Cliente Atual'].unique()

dados_matriz = []
for d in datas_range:
    mes_str = d.strftime("%Y-%m")
    for ass in lista_assessores:
        for f_abert in lista_faixas_abert:
            for f_atual in lista_faixas_atual:
                dados_matriz.append({
                    "Mes": mes_str,
                    "Assessor": ass,
                    "Faixa Cliente Abertura": f_abert,
                    "Faixa Cliente Atual": f_atual
                })

df_matriz = pd.DataFrame(dados_matriz)

# Merge com os dados reais
df_final = pd.merge(df_matriz, df_agg, on=['Mes', 'Assessor', 'Faixa Cliente Abertura', 'Faixa Cliente Atual'], how='left')
df_final.fillna(0, inplace=True)

# Filtra apenas assessores ativos na tabela 'times'
times = AzureLoader.ler_tabela("times_nova_empresa", schema=SCHEMA_DEFAULT)
if not times.empty:
    df_final = df_final[df_final['Assessor'].isin(times['Assessor'].str.upper())]

# Upload Tabela Agregada
AzureLoader.enviar_df(df_final, "abertura_de_conta", if_exists='replace', schema=SCHEMA_DEFAULT)

[AzureLoader] Lendo: dbo.times_nova_empresa...
[AzureLoader] Subindo tabela: dbo.abertura_de_conta (1792 linhas)...
[AzureLoader] Chunksize calculado: 298 linhas/lote (Colunas: 7).
[AzureLoader] Concluido: abertura_de_conta atualizada.


In [8]:
primeiro_dia_semana = (datetime.today() - timedelta(days = datetime.today().weekday())).strftime("%Y-%m-%d")

contas_semana = base_filtrada[base_filtrada['Data Vínculo'] >= primeiro_dia_semana].copy()

# Busca info extra de clientes
infos_clientes = AzureLoader.ler_tabela("infos_clientes", schema=SCHEMA_DEFAULT)
if not infos_clientes.empty:
    infos_clientes.rename(columns={"Tipo":"Tipo de conta"}, inplace=True)
    infos_clientes['Conta'] = infos_clientes['Conta'].astype(str).str.strip()
    
    contas_semana = contas_semana.merge(infos_clientes[['Conta', 'Tipo de conta', 'Profissão / Setor', 'IDADE']], on='Conta', how='left')

# Seleciona colunas finais
cols_export = ['Data de Abertura', 'Data Vínculo', 'Conta', 'Nome', 'Assessor', 'Tipo', 'PL Abertura', 'PL Atual', 'Tipo de conta', 'Profissão / Setor', 'IDADE']
# Garante que existem
cols_export_finais = [c for c in cols_export if c in contas_semana.columns]
contas_semana = contas_semana[cols_export_finais]

# Salva Excel
caminho_export = r"C:\Scripts\backups_atria\diarios\arquivos_banco\scripts\relatorios\entradas_e_saidas\contas_novas_semana.xlsx"
os.makedirs(os.path.dirname(caminho_export), exist_ok=True) # Cria pasta se não existir

contas_semana.to_excel(caminho_export, header=True, index=False)
print(f"   -> Relatório salvo em: {caminho_export}")

print("\n--- Script Finalizado com Sucesso ---")

[AzureLoader] Lendo: dbo.infos_clientes...
   -> Relatório salvo em: C:\Scripts\backups_atria\diarios\arquivos_banco\scripts\relatorios\entradas_e_saidas\contas_novas_semana.xlsx

--- Script Finalizado com Sucesso ---
